In [1]:
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
submit = pd.read_csv('gender_submission.csv')

In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 66.2+ KB


In [4]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare           417 non-null float64
Cabin          91 non-null object
Embarked       418 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 27.8+ KB


In [5]:
train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [6]:
test.describe()

,PassengerId,Pclass,Age,SibSp,Parch,Fare
count,418.000000,418.000000,332.000000,418.000000,418.000000,417.000000
mean,1100.500000,2.265550,30.272590,0.447368,0.392344,35.627188
std,120.810458,0.841838,14.181209,0.896760,0.981429,55.907576
min,892.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,996.250000,1.000000,21.000000,0.000000,0.000000,7.895800
50%,1100.500000,3.000000,27.000000,0.000000,0.000000,14.454200
75%,1204.750000,3.000000,39.000000,1.000000,0.000000,31.500000
max,1309.000000,3.000000,76.000000,8.000000,9.000000,512.329200


### 合併train、test資料

In [7]:
#合併train、test
data=train.append(test)

In [10]:
data

,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket
0,22.0,NaN,S,7.2500,"Braund, Mr. Owen Harris",0,1,3,male,1,0.0,A/5 21171
1,38.0,C85,C,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,2,1,female,1,1.0,PC 17599
2,26.0,NaN,S,7.9250,"Heikkinen, Miss. Laina",0,3,3,female,0,1.0,STON/O2. 3101282
3,35.0,C123,S,53.1000,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,4,1,female,1,1.0,113803
4,35.0,NaN,S,8.0500,"Allen, Mr. William Henry",0,5,3,male,0,0.0,373450
5,NaN,NaN,Q,8.4583,"Moran, Mr. James",0,6,3,male,0,0.0,330877
6,54.0,E46,S,51.8625,"McCarthy, Mr. Timothy J",0,7,1,male,0,0.0,17463
7,2.0,NaN,S,21.0750,"Palsson, Master. Gosta Leonard",1,8,3,male,3,0.0,349909
8,27.0,NaN,S,11.1333,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",2,9,3,female,0,1.0,347742
9,14.0,NaN,C,30.0708,"Nasser, Mrs. Nicholas (Adele Achem)",0,10,2,female,1,1.0,237736


In [9]:
#append合併之後，造成index重複問題，因此要將index重新設定
data.reset_index(inplace=True,drop=True)

#                    【資料分析】

## ------------------間斷資料分析----------------------

###  sns.countplot(data[' '],hue=data[' '])

In [ ]:
# 【生存率分析】
# 發現 死亡的比例:6成、生存的比例:4成
sns.countplot(data['Survived'])  

In [ ]:
# 【艙等】 跟生存率的關係
# 發現 生存率最高排序=>1艙等、2艙等、3艙等
sns.countplot(data['Pclass'],hue=data['Survived'])

In [ ]:
# 【性別】跟生存率的關係，發現女生生存率是男生的好幾倍。
sns.countplot(data['Sex'],hue=data['Survived'])

In [ ]:
# 【出發港口】跟生存率  s港出發的都比較容易死亡
sns.countplot(data['Embarked'],hue=data['Survived'])

## ------------------連續資料分析-----------------------

### sns.FacetGrid(data, col='間斷資料欄位')

### 資料名.map(sns.distplot, '連續資料欄位', kde=False)

In [ ]:
# 【年齡】跟生存率    年齡小的存活比例高
# sns.FacetGrid g.map sns.distplot  
g = sns.FacetGrid(data, col='Survived')
g.map(sns.distplot, 'Age', kde=False)

In [ ]:
# 【票價】、生存率  票價低的乘客死亡率高
g = sns.FacetGrid(data, col='Survived')
g.map(sns.distplot, 'Fare', kde=False)

In [ ]:
# 【父母＋小孩的數量】、生存率  
# 沒有跟父母小孩一起來的 生存率低
g = sns.FacetGrid(data, col='Survived')
g.map(sns.distplot, 'Parch', kde=False)

In [ ]:
# 【兄弟姊妹＋丈夫妻子的數量】、生存率  
# 沒有帶兄弟姊妹＋丈夫妻子一起來的 生存率低
g = sns.FacetGrid(data, col='Survived')
g.map(sns.distplot, 'SibSp', kde=False)

In [30]:
# 把家人加在同一欄位
data['Family_Size']=data['Parch']+data['SibSp']

In [ ]:
g = sns.FacetGrid(data, col='Survived')
g.map(sns.distplot, 'Family_Size', kde=False)

# 【特徵分析】

## 1. ----------------缺失值處理--------------------------

In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 12 columns):
Age            1309 non-null float64
Cabin          295 non-null object
Embarked       1309 non-null object
Fare           1309 non-null float64
Name           1309 non-null object
Parch          1309 non-null int64
PassengerId    1309 non-null int64
Pclass         1309 non-null int64
Sex            1309 non-null object
SibSp          1309 non-null int64
Survived       1309 non-null object
Ticket         1309 non-null object
dtypes: float64(2), int64(4), object(6)
memory usage: 92.1+ KB


In [12]:
data['Age'].fillna(data['Age'].mean(), inplace=True)
data['Embarked'].fillna('S', inplace=True)
data['Fare'].fillna(data['Fare'].mean(), inplace=True)
data['Survived'].fillna('0', inplace=True)

In [ ]:
data.head()

In [ ]:
data.info()

## 2. ---------------------取出人稱謂---------------------

### data['Name'].str.split(', ',expand=True)[0 or 1]

In [14]:
# 分割 name ,
# expand=True  ?????????????????????????????
# [1]          分割兩群之後，保留(前:0 或 後:1)
data['Title1']=data['Name'].str.split(', ',expand=True)[1]

In [ ]:
data.head()

In [ ]:
data['Name'].str.split(', ',expand=True).head(3)  #(看資料第幾筆)

In [ ]:
data['Title1'].head(3)

In [15]:
# 分割 name .
data['Title1']=data['Title1'].str.split('. ',expand=True)[0]

In [ ]:
data['Title1'].head(3)

In [16]:
#建立稱謂清單
data['Title1'].unique()

array(['Mr', 'Mrs', 'Miss', 'Master', 'Don', 'Rev', 'Dr', 'Mme', 'Ms',
       'Major', 'Lady', 'Sir', 'Mlle', 'Col', 'Capt', 'th', 'Jonkheer',
       'Dona'], dtype=object)

## ------分析 :人稱謂、性別、生存率------

### pd.crosstab(data['Title1'],data['Sex']).T.style.background_gradient(cmap='summer_r')

In [ ]:
pd.crosstab(data['Title1'],data['Sex']).T.style.background_gradient(cmap='summer_r')

In [ ]:
pd.crosstab(data['Title1'],data['Survived']).T.style.background_gradient(cmap='summer_r')

In [ ]:
# groupby稱謂，計算平均年齡
data.groupby(['Title1'])['Age'].mean()

In [ ]:
data.groupby(['Title1','Pclass'])['Age'].mean()

##  覆寫少數稱謂成 Miss Mrs Mr。
#### 某些稱謂的乘客佔少數，多了某些稱謂欄，對於機器學習模型，不是好事。故將少數稱謂覆寫。

### data['Title2']=data['Title1'].replace([name...] , [name1...])

In [17]:
data['Title2']=data['Title1'].replace(['Mlle','Mme','Ms','Dr','Major','Lady','th','Don','Rev','Sir','Col','Capt','Jonkheer','Dona'],
                                     ['Miss','Mrs','Miss','Mr','Mr','Mrs','Mrs','Mr','Mr','Mr','Mr','Mr','Mr','Mrs'])

In [18]:
data['Title2'].unique()

array(['Mr', 'Mrs', 'Miss', 'Master'], dtype=object)

In [ ]:
pd.crosstab(data['Title2'],data['Sex']).T.style.background_gradient(cmap='summer_r')

In [ ]:
pd.crosstab(data['Title2'],data['Survived']).T.style.background_gradient(cmap='summer_r')

In [ ]:
data.groupby('Title2')['Age'].mean()

In [ ]:
data.groupby(['Title2','Pclass'])['Age'].mean()

In [ ]:
list(data.groupby(['Title2','Pclass'])['Age'].mean().iteritems())[:3]

In [19]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 14 columns):
Age            1309 non-null float64
Cabin          295 non-null object
Embarked       1309 non-null object
Fare           1309 non-null float64
Name           1309 non-null object
Parch          1309 non-null int64
PassengerId    1309 non-null int64
Pclass         1309 non-null int64
Sex            1309 non-null object
SibSp          1309 non-null int64
Survived       1309 non-null object
Ticket         1309 non-null object
Title1         1309 non-null object
Title2         1309 non-null object
dtypes: float64(2), int64(4), object(8)
memory usage: 102.3+ KB


## 3. ----------取出票號 前面英文(房間)------------

### .apply(lambda x : x.replace(".","").replace("/","").strip().split(' ')[0] if not x.isdigit() else 'X')

In [20]:
data['Ticket_info'] = data['Ticket'].apply(lambda x : x.replace(".","").replace("/","").strip().split(' ')[0] if not x.isdigit() else 'X')

In [21]:
data['Ticket_info'].unique()

array(['A5', 'PC', 'STONO2', 'X', 'PP', 'CA', 'SCParis', 'SCA4', 'A4',
       'SP', 'SOC', 'WC', 'SOTONOQ', 'WEP', 'STONO', 'C', 'SCPARIS',
       'SOP', 'Fa', 'LINE', 'FCC', 'SWPP', 'SCOW', 'PPP', 'SC', 'SCAH',
       'AS', 'SOPP', 'FC', 'SOTONO2', 'CASOTON', 'SCA3', 'STONOQ', 'AQ4',
       'A', 'LP', 'AQ3'], dtype=object)

In [ ]:
# Ticket_info(房間)為間斷資料
sns.countplot(data['Ticket_info'], hue=data['Survived'])

## 3.------------------- 客艙、生存率 -------------------

In [ ]:
data['Cabin'].head(10)

In [22]:
# 選出客艙第一個字母 ，缺值則改為'NoCabin'
data["Cabin"] = data['Cabin'].apply(lambda x : str(x)[0] if not pd.isnull(x) else 'NoCabin')

In [23]:
# 建立客艙清單
data["Cabin"].unique()

array(['NoCabin', 'C', 'E', 'G', 'D', 'A', 'B', 'F', 'T'], dtype=object)

In [ ]:
# Cabin(客艙)為間斷資料
sns.countplot(data['Cabin'], hue=data['Survived'])

#  【將類別資料轉為整數】

In [24]:
data.head(5)

,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket,Title1,Title2,Ticket_info
0,22.0,NoCabin,S,7.2500,"Braund, Mr. Owen Harris",0,1,3,male,1,0,A/5 21171,Mr,Mr,A5
1,38.0,C,C,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,2,1,female,1,1,PC 17599,Mrs,Mrs,PC
2,26.0,NoCabin,S,7.9250,"Heikkinen, Miss. Laina",0,3,3,female,0,1,STON/O2. 3101282,Miss,Miss,STONO2
3,35.0,C,S,53.1000,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,4,1,female,1,1,113803,Mrs,Mrs,X
4,35.0,NoCabin,S,8.0500,"Allen, Mr. William Henry",0,5,3,male,0,0,373450,Mr,Mr,X


In [25]:
data['Cabin'] = data['Cabin'].astype('category').cat.codes
data['Embarked'] = data['Embarked'].astype('category').cat.codes
data['Sex'] = data['Sex'].astype('category').cat.codes
data['Pclass'] = data['Pclass'].astype('category').cat.codes
data['Title1'] = data['Title1'].astype('category').cat.codes
data['Title2'] = data['Title2'].astype('category').cat.codes
data['Ticket_info'] = data['Ticket_info'].astype('category').cat.codes

In [26]:
data.head(5)

,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket,Title1,Title2,Ticket_info
0,22.0,7,2,7.2500,"Braund, Mr. Owen Harris",0,1,2,1,1,0,A/5 21171,12,2,2
1,38.0,2,0,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,2,0,0,1,1,PC 17599,13,3,14
2,26.0,7,2,7.9250,"Heikkinen, Miss. Laina",0,3,2,0,0,1,STON/O2. 3101282,9,1,31
3,35.0,2,2,53.1000,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,4,0,0,1,1,113803,13,3,36
4,35.0,7,2,8.0500,"Allen, Mr. William Henry",0,5,2,1,0,0,373450,12,2,36


In [27]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 15 columns):
Age            1309 non-null float64
Cabin          1309 non-null int8
Embarked       1309 non-null int8
Fare           1309 non-null float64
Name           1309 non-null object
Parch          1309 non-null int64
PassengerId    1309 non-null int64
Pclass         1309 non-null int8
Sex            1309 non-null int8
SibSp          1309 non-null int64
Survived       1309 non-null object
Ticket         1309 non-null object
Title1         1309 non-null int8
Title2         1309 non-null int8
Ticket_info    1309 non-null int8
dtypes: float64(2), int64(3), int8(7), object(3)
memory usage: 75.5+ KB


#  【拆出訓練集、測試集】

## 單一Survived欄位為 pandas.core.series.Series ，需將Y轉成整數 Y=Y.astype(int)

In [109]:
selected_features = ['PassengerId','Age','Cabin','Embarked','Fare','Parch','Pclass','Sex','Parch','SibSp','Family_Size','Title2','Ticket_info']

In [110]:
# ------------------------------------------ X ------------------------------------------
X = data[selected_features]

In [111]:
X.head(3)

,PassengerId,Age,Cabin,Embarked,Fare,Parch,Pclass,Sex,Parch,SibSp,Family_Size,Title2,Ticket_info
0,1,22.0,7,2,7.2500,0,2,1,0,1,1,2,2
1,2,38.0,2,0,71.2833,0,0,0,0,1,1,3,14
2,3,26.0,7,2,7.9250,0,2,0,0,0,0,1,31


In [112]:
X=X.astype(int)

In [113]:
type(X)

pandas.core.frame.DataFrame

In [114]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 13 columns):
PassengerId    1309 non-null int32
Age            1309 non-null int32
Cabin          1309 non-null int32
Embarked       1309 non-null int32
Fare           1309 non-null int32
Parch          1309 non-null int32
Pclass         1309 non-null int32
Sex            1309 non-null int32
Parch          1309 non-null int32
SibSp          1309 non-null int32
Family_Size    1309 non-null int32
Title2         1309 non-null int32
Ticket_info    1309 non-null int32
dtypes: int32(13)
memory usage: 66.5 KB


In [115]:
# ------------------------------------------ Y ------------------------------------------
Y = data['Survived']

In [116]:
Y.head(3)

0    0
1    1
2    1
Name: Survived, dtype: object

In [117]:
Y=Y.astype(int)

In [118]:
type(Y)

pandas.core.series.Series

In [119]:
Y.info()

AttributeError: 'Series' object has no attribute 'info'

In [120]:
from sklearn.model_selection import train_test_split

In [121]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=9487)

In [122]:
# ------------------------------------ Y、y_train 都要改成'整數' ------------------------------------
y_train=y_train.astype(int)

# 【使用隨機森林預測】

In [123]:
# 建立 Random Forest 模組
rfc = RandomForestClassifier() 

In [ ]:
#cross_val_score(rfc, X_train, y_train, cv=10).mean()

In [124]:
rfc.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [125]:
survived_predict = rfc.predict(x_test)

In [127]:
submission=pd.DataFrame({'PassengerId':test['PassengerId'], 'Survived':survived_predict})

ValueError: array length 262 does not match index length 418

In [128]:
submission.to_csv('submission.csv', index=False)

NameError: name 'submission' is not defined

# 【使用隨機森林來推測年齡】

In [ ]:
dataAgeNull = data[data["Age"].isnull()]
dataAgeNotNull = data[data["Age"].notnull()]
remove_outlier = dataAgeNotNull[(np.abs(dataAgeNotNull["Fare"]-dataAgeNotNull["Fare"].mean())>(4*dataAgeNotNull["Fare"].std()))|
                      (np.abs(dataAgeNotNull["Family_Size"]-dataAgeNotNull["Family_Size"].mean())>(4*dataAgeNotNull["Family_Size"].std()))                     
                     ]
rfModel_age = RandomForestRegressor(n_estimators=2000,random_state=42)
ageColumns = ['Embarked', 'Fare', 'Pclass', 'Sex', 'Family_Size', 'Title1', 'Title2','Cabin','Ticket_info']
rfModel_age.fit(remove_outlier[ageColumns], remove_outlier["Age"])

ageNullValues = rfModel_age.predict(X= dataAgeNull[ageColumns])
dataAgeNull.loc[:,"Age"] = ageNullValues
data = dataAgeNull.append(dataAgeNotNull)
data.reset_index(inplace=True, drop=True)

In [ ]:
dataTrain = data[pd.notnull(data['Survived'])].sort_values(by=["PassengerId"])
dataTest = data[~pd.notnull(data['Survived'])].sort_values(by=["PassengerId"])

In [ ]:
dataTrain.columns

In [ ]:
dataTrain = dataTrain[['Survived', 'Age', 'Embarked', 'Fare',  'Pclass', 'Sex', 'Family_Size', 'Title2','Ticket_info','Cabin']]
dataTest = dataTest[['Age', 'Embarked', 'Fare', 'Pclass', 'Sex', 'Family_Size', 'Title2','Ticket_info','Cabin']]

In [ ]:
dataTrain

# 【Random Forest　預測存活率】

In [ ]:
from sklearn.ensemble import RandomForestClassifier
 
rf = RandomForestClassifier(criterion='gini', 
                             n_estimators=1000,
                             min_samples_split=12,
                             min_samples_leaf=1,
                             oob_score=True,
                             random_state=1,
                             n_jobs=-1) 

rf.fit(dataTrain.iloc[:, 1:], dataTrain.iloc[:, 0])
print("%.4f" % rf.oob_score_)

In [ ]:
pd.concat((pd.DataFrame(dataTrain.iloc[:, 1:].columns, columns = ['variable']), 
           pd.DataFrame(rf.feature_importances_, columns = ['importance'])), 
          axis = 1).sort_values(by='importance', ascending = False)[:20]

# 【將欲提交至kaggle的檔案寫出】

In [ ]:
submission=pd.DataFrame({'PassengerId':test['PassengerId'], 'Survived':survived_predict})

In [ ]:
rf_res =  rf.predict(dataTest)
submit['Survived'] = rf_res
submit['Survived'] = submit['Survived'].astype(int)
submit.to_csv('submit.csv', index= False)

In [ ]:
submit